## Nested Anova

*This was the nested anova which I implemented. However, it was not the correct analysis to make. Rather than throwing it away, I wanted to keep it for future reference.*

*Here are the notes I made at the time:*

Because we have 50-odd individuals, each of which carried out 50+ samples, I thought I could carry out a nested anova. Here, there are two factors - one is the experimental condition, the second is the individual. Compare with Steve McKillup's example on p 224. He has prawns and ponds. His factors are treatment (analogous to our condition) and pond (analogous to our individual). Compare also with Jerrold H. Zar, Chapter 15, p 307: He has these factors: Drug - the "group" (anal. to Steve's treatment or our condition) and Source - the "subgroup" (anal. to Steve's pond and our individual).

Our condition is a "fixed effects factor". Our individual is a "random effects factor".

***However, because we apply each condition to each individual, we actually have a "two factor with replication" design with one fixed factor (condition) and one random factor (individual)***

In [1]:
# Import the data, which should be available in Matlab v7 format:
import scipy.io as sio
mat_workspace = sio.loadmat('AllData/fnames.mat')
# fnames is used throughout the rest of this notebook, so this section needs to be evaluated!
fnames = mat_workspace['fnames']

In [2]:
import numpy as np
from random import randint

def getfnameid (filename):
    # idarr[1] is the ID, idarr[0] is the experimenter, idarr[3] is the dated filename.
    idarr = filename.split('/')
    return idarr[1]

# From the condition string, return an index for the condition. 0 is 
# "No Distractor trial", 1 is "Synchronous Distractor trail", 2 is
# "Asynchronous Distractor trial".
def getcondition (condition_string):
    condition_index = -1
    if 'No Dist' in condition_string:
        condition_index = 0
    elif 'Synchro' in condition_string:
        condition_index = 1
    elif 'Asynchr' in condition_string:
        condition_index = 2
    return condition_index

# A Single Factor ANOVA calculation for three datasets
def group_anova(no_dist_latencies,sync_latencies,async_latencies):

    all_latencies = np.concatenate((no_dist_latencies, sync_latencies, async_latencies))
        
    # Compute grand mean
    grand_mean = all_latencies.mean()
    #print 'Grand mean:',grand_mean,'(',all_latencies.var(ddof=1),') not ',all_latencies.var(ddof=0)
        
    # Compute within-group variance
    tmp1 = all_latencies
    np.power(tmp1, 2)
    within_group_dof = all_latencies.size-3
    within_group_variance = tmp1.sum()/within_group_dof
    #print 'within_group_variance',within_group_variance
    
    nodist_mean = no_dist_latencies.mean()
    sync_mean = sync_latencies.mean()
    async_mean = async_latencies.mean()
    
    # Compute amoung-group variance
    tmp1 = np.power (grand_mean - nodist_mean, 2)*no_dist_latencies.size
    tmp2 = np.power (grand_mean - sync_mean, 2)*sync_latencies.size
    tmp3 = np.power (grand_mean - async_mean, 2)*async_latencies.size
    sosquares = tmp1 + tmp2 + tmp3
    between_group_dof = 2 # 3 conditions => 3 groups, so 3-1 DOF
    between_group_variance = sosquares / between_group_dof
    #print 'between_group_variance',between_group_variance
        
    # Now compute the F ratio
    F = between_group_variance/within_group_variance
        
    # Lastly, what's the probability for this?
    P = 1-special.fdtr(between_group_dof,within_group_dof,F)
        
    return (F, between_group_dof, within_group_dof, P)

# Take n sub-samples from distn
def subsample (distn, n):
    counter = 0
    subsamp = []
    for s in distn:
        if np.random.uniform()>0.5:
            subsamp = np.append(subsamp, s)
            counter = counter + 1
        if counter >= n:
            break
    return subsamp

# Libs used in class individual
from scipy import special
from scipy import stats
import statsmodels.api as sm
from matplotlib import pyplot as plt
import matplotlib.lines as mlines

# Class for an individual's data.
class individual:
    def __init__(self, subj_id):
        self.subj_id = subj_id;
        self.no_dist_latencies = np.ndarray(1)
        self.sync_latencies = np.ndarray(1)
        self.async_latencies = np.ndarray(1)
        self._nodist_mean = -1
        self._sync_mean = -1
        self._async_mean = -1
        self._nodist_std = -1
        self._sync_std = -1
        self._async_std = -1

    # Compute ANOVA for this individual
    def anova(self):
        F, between_group_dof, within_group_dof, P = group_anova (self.no_dist_latencies, self.sync_latencies, self.async_latencies)
        return (F, between_group_dof, within_group_dof, P)

    def reportmeans (self):
        print "Mean(SD): No distr: {0:.2f} ({1:.2f}) Sync: {2:.2f} ({3:.2f}) Async: {4:.2f} ({5:.2f})".format(self.nodist_mean(), self.nodist_std(), self.sync_mean(), self.sync_std(), self.async_mean(), self.async_std())                

    # Batch up all data in a form suitable for statsmodel's MultiComparison class. This means
    # concatenating the ND, SD & AD data intoa single array, and making a "label" array to match.
    def getMultiComparisonData (self):
        d = np.hstack((self.no_dist_latencies,self.sync_latencies,self.async_latencies))

        nd_labels = np.ndarray(shape=(self.no_dist_latencies.size,), dtype=object)
        nd_labels.fill('ND')

        sd_labels = np.ndarray(shape=(self.sync_latencies.size,), dtype=object)
        sd_labels.fill('SD')

        ad_labels = np.ndarray(shape=(self.async_latencies.size,), dtype=object)
        ad_labels.fill('AD')

        l = np.hstack((nd_labels,sd_labels,ad_labels))

        # d is the data array, l is the label array.
        return (d, l)

    # Do a full set of graphs to show the normality of the data. Show QQ plots,
    # histograms of the distributions and results of Shapiro-Wilks tests for comparison.
    # Pass in the significance level for the S-W test.
    def shownormality (self, alpha):
        f, axarr = plt.subplots(3, 2)
        
        #ax1.set_title('QQ plots')
        fig1 = sm.qqplot(self.no_dist_latencies, fit=True, line='45',ax=axarr[0,0])
        fig2 = sm.qqplot(self.sync_latencies, fit=True, line='45',ax=axarr[1,0])
        fig3 = sm.qqplot(self.async_latencies, fit=True, line='45', ax=axarr[2,0])
        axarr[0,0].set_title('QQ Plots')
        
        W, p = stats.shapiro (subsample(self.no_dist_latencies, 25))
        isNormal = False
        if p > alpha:
            isNormal = True
        sw = 'ND. Mean/SD:{2:.2f}/{3:.2f} W={0:.2f}, p={1:.2f} (Normal:{4})'.format(W,p,self.nodist_mean(),self.nodist_std(),isNormal)
        axarr[0,1].hist(self.no_dist_latencies, bins=20, label=sw)
        axarr[0,1].legend(prop={'size':9})
        axarr[0,1].set_title('Dist\'ns with Shapiro-Wilks stats')
        
        W, p = stats.shapiro (subsample(self.sync_latencies, 25))
        isNormal = False
        if p > alpha:
            isNormal = True
        sw = 'SD. Mean/SD:{2:.2f}/{3:.2f} W={0:.2f}, p={1:.2f} (Normal:{4})'.format(W,p,self.sync_mean(),self.sync_std(),isNormal)
        axarr[1,1].hist(self.sync_latencies, bins=20, label=sw)
        axarr[1,1].legend(prop={'size':9})

        W, p = stats.shapiro (subsample(self.async_latencies, 25))
        isNormal = False
        if p > alpha:
            isNormal = True
        sw = 'AD. Mean/SD:{2:.2f}/{3:.2f} W={0:.2f}, p={1:.2f} (Normal:{4})'.format(W,p,self.async_mean(),self.async_std(),isNormal)
        axarr[2,1].hist(self.async_latencies, bins=20, label=sw)
        axarr[2,1].legend(prop={'size':9})

        # Fine-tune figure; make subplots close to each other and hide x ticks for
        # all but bottom plot.
        f.subplots_adjust(hspace=0)
        plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)
        
        plt.show()
        
    # Apply Shapiro-Wilk test. Null hypothesis is that the data are normally
    # distributed. If p < alpha then null hypothesis must be rejected and data
    # cannot be considered to be normally distributed.
    def shapiroWilk (self, condition, alpha):

        W = -1
        p = -1
        isNormal = False

        if condition == 0:
            W, p = stats.shapiro (subsample(self.no_dist_latencies, 25))
        elif condition == 1:
            W, p = stats.shapiro (subsample(self.sync_latencies, 25))
        elif condition == 2:
            W, p = stats.shapiro (subsample(self.async_latencies, 25))
        # else leave W,p,isNormal with default values

        if p > alpha:
            isNormal = True

        return W, p, isNormal
    
    # Do a Quantile-Quantile plot to compare against normal distribution
    def qqplot (self):
        f, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, sharey=True)
        ax1.set_title('QQ plots')
        fig1 = sm.qqplot(self.no_dist_latencies, fit=True, line='45',ax=ax1)
        fig2 = sm.qqplot(self.sync_latencies, fit=True, line='45',ax=ax2)
        fig3 = sm.qqplot(self.async_latencies, fit=True, line='45', ax=ax3)
        # Fine-tune figure; make subplots close to each other and hide x ticks for
        # all but bottom plot.
        f.subplots_adjust(hspace=0)
        plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)
        plt.show()
        return f

    def excludeOutliers (self, num_sds):
        #print "Removing outliers", num_sds, "SDs away from the mean"
        r = 0
        while r < self.no_dist_latencies.size:
            upperlimit = self.nodist_mean() + self.nodist_std()*num_sds
            lowerlimit = self.nodist_mean() - self.nodist_std()*num_sds
            if self.no_dist_latencies[r] > upperlimit or self.no_dist_latencies[r] < lowerlimit:
                # outlier, so delete
                self.no_dist_latencies = np.delete (self.no_dist_latencies, r, 0)
                r = r - 1
            r = r + 1

        r = 0
        while r < self.sync_latencies.size:
            if self.sync_latencies[r] > (self.sync_mean() + self.sync_std()*num_sds) or self.sync_latencies[r] < (self.sync_mean() - self.sync_std()*num_sds):
                self.sync_latencies = np.delete (self.sync_latencies, r, 0)
                r = r - 1
            r = r + 1

        r = 0
        while r < self.async_latencies.size:
            if self.async_latencies[r] > (self.async_mean() + self.async_std()*num_sds) or self.async_latencies[r] < (self.async_mean() - self.async_std()*num_sds):
                self.async_latencies = np.delete (self.async_latencies, r, 0)
                r = r - 1
            r = r + 1

    def randomly_subsample_data (self, num_data):
        while len(self.no_dist_latencies) > num_data:
            remove_this = randint (0,len(self.no_dist_latencies)-1)
            self.no_dist_latencies = np.delete(self.no_dist_latencies, remove_this)
        while len(self.sync_latencies) > num_data:
            remove_this = randint (0,len(self.sync_latencies)-1)
            self.sync_latencies = np.delete(self.sync_latencies, remove_this)
        while len(self.async_latencies) > num_data:
            remove_this = randint (0,len(self.async_latencies)-1)
            self.async_latencies = np.delete(self.async_latencies, remove_this)
 
    def graph1(self):
        print 'Showing graph for ', ind.subj_id
        means = (self.nodist_mean(), self.sync_mean(), self.async_mean())
        stds = (self.nodist_std(), self.sync_std(), self.async_std())
        index = np.arange(3)
        opacity = 0.4
        error_config = {'ecolor': '0.3'}
        rects1 = plt.bar(index, means, 0.2,
                 alpha=opacity,
                 color='b',
                 yerr=stds,
                 error_kw=error_config,
                 label=ind.subj_id)
        # Now draw the points on a scatter graph
        nodist_cond_x = np.zeros(self.no_dist_latencies.size)
        sync_cond_x = np.ones(self.sync_latencies.size)
        async_cond_x = 2*np.ones(self.async_latencies.size)
        nodist_pts = plt.scatter(nodist_cond_x, self.no_dist_latencies)
        sync_pts = plt.scatter(sync_cond_x, self.sync_latencies)
        async_pts = plt.scatter(async_cond_x, self.async_latencies)
        plt.xlabel('Condition 0:ND 1:S 2:AS')
        plt.ylabel('Latency (ms)')
        plt.title(self.subj_id)
        plt.show()
        return

    # Compute the sum of the squared displacements from the mean for all three conditions
    def sumofsquare_displacements_all_from_value(self, value):
        sos = self.sumofsquare_displacements_from_value(0,value) + self.sumofsquare_displacements_from_value(1,value) + self.sumofsquare_displacements_from_value(2,value)
        return sos

    # Compute the sum of the squared displacements from the mean for all three conditions
    def sumofsquare_displacements_all(self):
        sos = self.sumofsquare_displacements(0) + self.sumofsquare_displacements(1) + self.sumofsquare_displacements(2)
        return sos

    # Compute the sum of the squared displacements from the mean for the given condition
    def sumofsquare_displacements(self, condition):
        if condition == 0:
            mn = self.nodist_mean()
            squares = np.power((self.no_dist_latencies - mn), 2)
        elif condition == 1:
            mn = self.sync_mean()
            squares = np.power((self.sync_latencies - mn), 2)
        else: # condition 2
            mn = self.async_mean()
            squares = np.power((self.async_latencies - mn), 2)
        sos = np.sum(squares)
        return sos
    
    # Compute the sum of the squared displacements from the mean for the given condition
    def sumofsquare_displacements_from_value(self, condition, value):
        if condition == 0:
            squares = np.power((self.no_dist_latencies - value), 2)
            # Verification of this method:
            #squares_alt = 0
            #for i in self.no_dist_latencies:
            #    imv = i - value
            #    squares_alt += imv*imv
            #print 'nd sum of squares:',np.sum(squares),'squares_alt:',squares_alt
        elif condition == 1:
            squares = np.power((self.sync_latencies - value), 2)
        else: # condition 2
            squares = np.power((self.async_latencies - value), 2)
        sos = np.sum(squares)
        return sos

    def num_replicates_all(self):
        n = self.num_replicates(0) + self.num_replicates(1) + self.num_replicates(2)
        return n

    def num_replicates(self, condition):
        if condition == 0:
            return self.no_dist_latencies.size
        elif condition == 1:
            return self.sync_latencies.size
        else: # condition 2
            return self.async_latencies.size

    def nodist_mean(self):
        self._nodist_mean = self.no_dist_latencies.mean()
        return self._nodist_mean
        # This didn't work yet:
        #print "nodist_mean called self._nodist_mean==",self._nodist_mean
        #if self.no_dist_latencies.size == 1
        #    return self._nodist_mean
        #if self._nodist_mean == -1:
        #    self._nodist_mean = self.no_dist_latencies.mean()
        #return self._nodist_mean
        
    def sync_mean(self):
        self._sync_mean = self.sync_latencies.mean()
        return self._sync_mean

    def async_mean(self):
        self._async_mean = self.async_latencies.mean()
        return self._async_mean

    def overall_mean(self):
        all_latencies = np.concatenate((self.no_dist_latencies, self.sync_latencies, self.async_latencies))
        return all_latencies.mean()

    def overall_std(self):
        all_latencies = np.concatenate((self.no_dist_latencies, self.sync_latencies, self.async_latencies))
        return all_latencies.std()
    
    def nodist_std(self):
        self._nodist_std = self.no_dist_latencies.std()
        return self._nodist_std
        
    def sync_std(self):
        self._sync_std = self.sync_latencies.std()
        return self._sync_std

    def async_std(self):
        self._async_std = self.async_latencies.std()
        return self._async_std

    def __str__(self):
        return "Data container for subject {0}".format(self.subj_id)

def readIndividuals():
    individuals = dict()
    # Extract the data from the raw format and collate it into individual
    # data containers, one per subject.
    for fname in zip(*fnames):
        # I'll use the subject ID as a key into output data structures
        subj_id = getfnameid(fname[0][0])

        # condition index is for the no distractor/sync distractor/async distractor
        condition_index = getcondition(fname[1][0,0][36][0])

        # Need ONE individual object for each subj_id.
        if subj_id not in individuals:
            individuals[subj_id] = individual(subj_id)

        latencies = fname[4] # Use table with python index 4 - "no move error", "target event".
        
        alldata = fname[2] # Contains all data, including move errors
        
        if condition_index == 0:
            individuals[subj_id].no_dist_latencies = latencies[:,4];
        elif condition_index == 1:
            individuals[subj_id].sync_latencies = latencies[:,4];
            # For sync, also read errors
            nerrs = 0
            ndistractors = 0
            #if subj_id == 'JS':
            #    print 'Sync'
            for d in alldata:
                #if subj_id == 'JS' and d[2] > 0.0:
                #    print d
                ndistractors += 1
                if d[2] > 0.0:
                    nerrs += 1
            individuals[subj_id].n_errors_per_distractor_sync = (float(nerrs) / float(ndistractors))

        elif condition_index == 2: 
            individuals[subj_id].async_latencies = latencies[:,4];
            # For async, also read errors
            nerrs = 0
            ndistractors = 0
            #if subj_id == 'JS':
            #    print 'Async'
            for d in alldata:
                #if subj_id == 'JS' and d[2] > 0.0:
                #    print d
                if d[2] > 0.0: # Count all errors for async
                    nerrs += 1
                if d[1] < 1.0:
                    ndistractors += 1
            if subj_id == 'JS':
                print 'Num movement errors:',nerrs,'Num distractor events:',ndistractors
            individuals[subj_id].n_errors_per_distractor_async = (float(nerrs) / float(ndistractors))
    
    return individuals


In [3]:
import numpy as np
def nested_anova(individuals, nodist_latencies, sync_latencies, async_latencies):

    all_latencies = np.concatenate((nodist_latencies, sync_latencies, async_latencies))
        
    # Compute/store grand mean and condition means:
    grand_mean = all_latencies.mean()
    nodist_mean = nodist_latencies.mean()
    sync_mean = sync_latencies.mean()
    async_mean = async_latencies.mean()

    # First. Error estimate from displacements within sub-groups (individuals)
    sos_error = 0
    dof_error = 0
    for ikey in individuals:
        i = individuals[ikey]
        sos_error = sos_error + i.sumofsquare_displacements_all()
        dof_error = dof_error + (i.num_replicates_all() - 1)
    meansquare_for_error = sos_error/dof_error
    
    # Second Ignore sub-groups (individuals) and compute meansquare for treatment, aka meansquare for group (condition) plus error.
    sq_displacement_nodist = np.power(grand_mean - nodist_mean, 2);
    sq_displacement_sync = np.power(grand_mean - sync_mean, 2);
    sq_displacement_async = np.power(grand_mean - async_mean, 2);
    dof_condition = 2
    meansquare_for_condition_plus_ind_plus_err = (sq_displacement_nodist + sq_displacement_sync + sq_displacement_async)/dof_condition
    
    # Third Displacement of subgroups from  group (condition) means. meansquare for subgroups plus error 
    sq_displacement_nodist = 0
    sq_displacement_sync = 0
    sq_displacement_async = 0
    for ikey in individuals:
        i = individuals[ikey]
        sq_displacement_nodist += np.power(nodist_mean-i.nodist_mean(), 2)
        sq_displacement_sync += np.power(sync_mean-i.sync_mean(), 2)
        sq_displacement_async += np.power(async_mean-i.async_mean(), 2)
    dof_individual = len(individuals) - 1
    meansquare_for_individuals_plus_err = (sq_displacement_nodist + sq_displacement_sync + sq_displacement_async)/dof_individual

    # Now compute the F statistic - this is just the variance ratio:
    # F for treatment aka condition
    F_condition =  meansquare_for_condition_plus_ind_plus_err / meansquare_for_individuals_plus_err
    # F for subgroup aka individual
    F_individual = meansquare_for_individuals_plus_err / meansquare_for_error
    
    # Lastly, what's the probability for this?
    P_condition = 1-special.fdtr(dof_condition, dof_individual, F_condition)
    P_individual = 1-special.fdtr(dof_individual, dof_error, F_individual)
    
    return (F_condition, F_individual, dof_condition, dof_individual, dof_error, P_condition, P_individual)

# Tukey testing? q = Condition mean A - Condition mean B / SEM where SEM = sqrt ( MS for individuals(conditions) / n )
#
# Implement examples from books to ensure that we get the same results.
def nested_tukey(individuals, nodist_latencies, sync_latencies, async_latencies):

    all_latencies = np.concatenate((nodist_latencies, sync_latencies, async_latencies))
        
    # Compute/store grand mean and condition means:
    grand_mean = all_latencies.mean()
    nodist_mean = nodist_latencies.mean()
    sync_mean = sync_latencies.mean()
    async_mean = async_latencies.mean()

    # For a Tukey on nested/hierarchical data, ignore sub-groups (individuals) and compute meansquare for treatment, aka meansquare for group (condition) plus error.
    sq_displacement_nodist = np.power(grand_mean - nodist_mean, 2);
    sq_displacement_sync = np.power(grand_mean - sync_mean, 2);
    sq_displacement_async = np.power(grand_mean - async_mean, 2);
    dof_condition = 2
    meansquare_for_condition_plus_ind_plus_err = (sq_displacement_nodist + sq_displacement_sync + sq_displacement_async)/dof_condition

    #sos_error = 0
    dof_error = 0
    for ikey in individuals:
        i = individuals[ikey]
        #sos_error = sos_error + i.sumofsquare_displacements_all()
        dof_error = dof_error + (i.num_replicates_all() - 1)
    #meansquare_for_error = sos_error/dof_error
    
    # se for standard error? se chosen to match Zar p 229
    se = np.sqrt(meansquare_for_condition_plus_ind_plus_err)
    
    # ND to SD Tukey
    tukey1 = np.abs(nodist_mean - sync_mean) / se
    tukey2 = np.abs(nodist_mean - async_mean) / se
    tukey3 = np.abs(sync_mean - async_mean) / se
    
    # What's q_alpha_v_k? alpha = 0.05; v= error degrees of freedom (dof_error); k=3 (k means)
    # From the book, that's q_0.05_inf_3, and it's 0.9539 X 1.588
    q_alpha_v_k = 1.588
    
    print 'ND vs SD:',tukey1,'Significant?',(tukey1>q_alpha_v_k)
    print 'ND vs AD:',tukey2,'Significant?',(tukey2>q_alpha_v_k)
    print 'SD vs AD:',tukey3,'Significant?',(tukey3>q_alpha_v_k)
    
    # if q > q_0.05_inf_3 then the null hypothesis that the distributions are the same is rejected.


In [4]:
# Read the fname data from the octave analysis into an array of individuals
individuals = readIndividuals()

# Three containers for every event latency
nodist_latencies = [];
sync_latencies = [];
async_latencies = [];

# Populate the three containers:
for i in individuals:
    ind = individuals[i]
    # Exclude outlier latencies on a per-individual basis:
    ind.excludeOutliers(2)
    nodist_latencies = np.concatenate((nodist_latencies, ind.no_dist_latencies))
    sync_latencies = np.concatenate((sync_latencies, ind.sync_latencies))
    async_latencies = np.concatenate((async_latencies, ind.async_latencies))

F_condition, F_individual, dof_condition, dof_individual, dof_error, P_condition, P_individual = nested_anova(individuals, nodist_latencies, sync_latencies, async_latencies)
print 'Individual variances:'
print 'F({0},{1})={2} P={3}'.format(dof_individual,dof_error,F_individual,P_individual)
print 'Condition variances:'
print 'F({0},{1})={2} P={3}'.format(dof_condition,dof_individual,F_condition,P_condition)

print '\nTukey testing:'
nested_tukey(individuals, nodist_latencies, sync_latencies, async_latencies)

Num movement errors: 29 Num distractor events: 44
Individual variances:
F(54,8082)=0.81370802512 P=0.833468414456
Condition variances:
F(2,54)=0.294519057518 P=0.746078601439

Tukey testing:
ND vs SD: 1.68582931963 Significant? True
ND vs AD: 1.76583679389 Significant? True
SD vs AD: 0.0800074742587 Significant? False


## Analysis of the nested Anova

The null hypothesis is that the variances were the result only of samples drawn from a single population showing random variation.

The alternative hypothesis is that the variances were the result of the samples being drawn from separate populations.

In the above, the P value is the probability of the null hypothesis holding.

So, P_cond = 0.75 says that there is a 75% chance that the null hypothesis is true and the condition variances for ND, sync and async samples were drawn from a single population.

P_ind = 0.996 says there is a 99.6 % probability that the individual variances were drawn from a single population.

**So this seems to be saying that there's no significant effect of the ND, sync or async conditions.**

In fact, I realised that this is not the correct analysis to make, because each condition is carried out by each individual. The students were right to select a One way repeated measures Anova (see below in this document).